# 数据操作

In [86]:
# 首先导入pytorch 虽然是pytorch 但是导入时是torch
import torch

张量表示一个数值组成的数组,这个数组可能有多个维度,是一个多维数组，它是标量、向量、矩阵的高维拓展

In [87]:
x = torch.arange(12)
x

我们可以通过张量的<font color=gray>shape</font>属性来访问张量(tensor)的形状和张量中元素的总数

In [88]:
x.shape

In [89]:
x.numel() # 获取元素的总数

要改变一个张量的形状而不改变元素数量和元素的值,可以调用reshape函数

In [90]:
x = x.reshape(3, 4)
x

通过提供包含数值的python列表(或者列表)来为所需张量中的每个元素赋予确定值

In [91]:
torch.tensor([[2,1,4,3],[1,2,3,4],[4,3,2,1]])

常见的标准运算符(+ - / * 或 **)都可以升级为<mark>按元素运算</mark>

In [92]:
x = torch.tensor([1.0,2,4,8])
y = torch.tensor([2,2,2,2])
x + y,x - y,x * y,x / y, x ** y

也可以把多个张量连结在一起

In [ ]:
X = torch.arange(12, dtype=torch.float32).reshape(3, 4)
Y = torch.tensor([[2.0,1,4,3],[1,2,3,4],[4,3,2,1]])
torch.cat((X,Y),dim = 0), torch.cat((X,Y), dim=1)
# dim 第n位

通过逻辑运算符构建二元张量

In [ ]:
X==Y

对张量中的所有元素进行求和会产生一个只有一个元素的张量

In [ ]:
X.sum()

tensor(66.)

就算形状不同,我们仍然可以调用 广播机制 (broadcasting mechanism)来直行按元素操作

In [ ]:
a = torch.arange(3).reshape((3,1)) # a复制成3*2的矩阵，b也如此
b = torch.arange(2).reshape((1,2))
a, b

(tensor([[0],
         [1],
         [2]]),
 tensor([[0, 1]]))

In [ ]:
a + b

tensor([[0, 1],
        [1, 2],
        [2, 3]])

可以用[-1]选择最后一个元素.[1:3]选择第二个和第三个元素

In [ ]:
X[-1],X[1:3] # 第一行和第二行的元素

(tensor([ 8.,  9., 10., 11.]),
 tensor([[ 4.,  5.,  6.,  7.],
         [ 8.,  9., 10., 11.]]))

除了读取之外,我们还可以通过指定索引将元素写入矩阵

In [ ]:
X[1,2] = 9
X

tensor([[ 0.,  1.,  2.,  3.],
        [ 4.,  5.,  9.,  7.],
        [ 8.,  9., 10., 11.]])

给多个元素赋值,只要索引所有元素然后给他们赋值就行了

In [ ]:
X[0:2,:] = 666
X

tensor([[666., 666., 666., 666.],
        [666., 666., 666., 666.],
        [  8.,   9.,  10.,  11.]])

运行一些操作可能会导致为新结果分配内存,python本身不用考虑内存问题,但是新操作可能会出现

In [ ]:
before = id(Y) # id就好像指针，标识号
Y = Y + X
id(Y) == before

False

执行原地操作
torch.zeros():返回一个形状为为size,类型为torch.dtype，里面的每一个值都是0的tensor

In [ ]:
Z = torch.zeros_like(Y) # 和Y的元素类型一样
print ('id(Z):', id(Z))
Z[:] = X + Y
print('id(Z):', id(Z))

id(Z): 2331881330208
id(Z): 2331881330208


如果在后续计算中没有重复使用x，我们也可以使用X[:] = X +Y或X += Y来减少操作的内存开销


In [ ]:
before = id(X)
X += Y
id(X) == before

True

转换为Numpy张量

In [ ]:
A = X.numpy()
B = torch.tensor(A)
type(A), type(B)

(numpy.ndarray, torch.Tensor)

将大小为1的张量转换为Python标量

In [ ]:
a = torch.tensor([3.5])
a, a.item(), float(a), int(a) # item 取出单元素张量的元素值并返回该值，保持该元素类型不变。

(tensor([3.5000]), 3.5, 3.5, 3)

# 数据预处理

创建一个人工数据集,并且存储在csv（逗号分隔值)文件

In [ ]:
import os

os.makedirs(os.path.join('.','data'),exist_ok=True)
data_file = os.path.join('.','data','house_tiny.csv')
with open(data_file,'w') as f:
    f.write('NumRooms,Alley,Price\n') # 列名字
    f.write('NA,Pave,127500\n') # 每行表示一个数据样本，NA未知
    f.write('2,NA,106010\n')
    f.write('4,NA,178100\n')
    f.write('NA,NA,140000\n')

出现问题：PermissionError: [WinError 5]
解决办法：将..改为.,权限问题

从创建的csv文件中加载原始数据集

In [107]:
# !pip install pandas
import pandas as pd
data = pd.read_csv(data_file)
print(data)

   NumRooms Alley   Price
0       NaN  Pave  127500
1       2.0   NaN  106010
2       4.0   NaN  178100
3       NaN   NaN  140000


In [108]:
import pandas as pd
data = pd.read_csv(data_file)
data

,NumRooms,Alley,Price
0,NaN,Pave,127500
1,2.0,NaN,106010
2,4.0,NaN,178100
3,NaN,NaN,140000


为了处理缺失的数据,典型方法：插值和删除,这里考虑插值

In [143]:
inputs, outputs = data.iloc[:,0:1],data.iloc[:,2]
# input里面存放所有行，第一和第二列，output里面放所有行和最后一列
# iloc[] 对数据进行位置索引，从而在数据表中提取出相应的数据。
inputs = inputs.fillna(inputs.mean())
print(inputs)

   NumRooms
0       3.0
1       2.0
2       4.0
3       3.0


对于inputs中类别值或者离散值,将NaN视为一个类别

In [133]:
inputs = pd.get_dummies(inputs, dummy_na=False)
print(inputs)

   NumRooms
0       3.0
1       2.0
2       4.0
3       3.0


inputs 和 outputs中所有条目都是数值,可以转换成张量

In [144]:
x, y = torch.tensor(inputs.values), torch.tensor(outputs.values)
x, y

(tensor([[3.],
         [2.],
         [4.],
         [3.]], dtype=torch.float64),
 tensor([127500, 106010, 178100, 140000]))